In [2]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [12]:
start_date = '2022-10-31'
end_date = '2022-11-13'
service = 'taxi'

In [13]:
pan_taxi = f"""
with base as (
select
    run_date,
    week,
    case 
    when lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
    when lifetime_stage = 'HOOK' then '2.HOOK'
    when lifetime_stage = 'DETOX' then '3.DETOX'
    when lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
    when lifetime_stage = 'COMMITTED' then '5.COMMITTED'
    when lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
    when lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
    when lifetime_stage = 'DORMANT' then '8.DORMANT'
    when lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
    end lifetime_stage,
    customer_id
from 
    (
    select
        run_date,
        (extract(week from cast(run_date as date)) + 1) as week,
        case
        when lower('{service}') = ('link') then link_lifetime_stage 
        when lower('{service}') = ('auto') then auto_lifetime_stage 
        else taxi_lifetime_stage end lifetime_stage,
        customer_id,
        count(*) records
    from datasets.iallocator_customer_segments 
    where 
        run_date between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '1' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '15' day), '%Y-%m-%d')
        and  date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and day_of_week(cast (run_date as date)) = 7
        and case
            when lower('{service}') = ('link') then link_lifetime_rides 
            when lower('{service}') = ('auto') then auto_lifetime_rides 
            else taxi_lifetime_rides end > 0 
    group by 1,2,3,4
    )
),

rf_daily_kpi as (
select
    base.lifetime_stage,
    base.week week_number,
    base.customer_id,
    a.*
from     
    (
        select
            customerid,
            extract(week from cast(day as date)) as week,
            date_format(date_trunc('week', cast(day as date)), '%Y-%m-%d') start_date,
            sum(gross_rides_daily) as gross_rides,
            sum(case when net_rides_daily > 0 then net_rides_daily end) as net_rides,
            sum(case when net_rides_daily > 0 then subscription_rides_daily end) as sub_rides,
            sum(case when net_rides_daily > 0 then burn_daily end) as burn,
            sum(case when subscription_rides_daily > 0 then burn_daily end) as subs_discount,
            sum(case when net_rides_daily > 0 then subtotal_daily end) as subtotal,
            sum(case
                when ao_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when ao_sessions_unique_daily = 0 and fe_sessions_unique_daily > 0 then fe_sessions_unique_daily
                when ao_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else ao_sessions_unique_daily end 
                ) as ao,
            sum(case
                when fe_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when fe_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else fe_sessions_unique_daily end 
                ) as fe,
            sum(case 
                when rr_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                else rr_sessions_unique_daily end ) as rr
        from 
        datasets.customer_rf_daily_kpi
        where
        day between  date_format(date_trunc('week', cast('{start_date}' as date)), '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y-%m-%d')
        and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and service_name in ('Auto', 'Link')
        group by 1,2,3
    ) as a
right join base on base.customer_id = a.customerid and base.week = a.week
),

orders as (
select
    week(date_parse(yyyymmdd,'%Y%m%d')) week,
    date_format(date_trunc('week',date_parse(yyyymmdd,'%Y%m%d')), '%Y-%m-%d') start_date,
    customer_id,
    order_id,
    city_name,
    discount,
    amount,
    sub_total,
    service_obj_service_name,
    order_status
from orders.order_logs_snapshot
where 
    yyyymmdd between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '14' day,'%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '28' day), '%Y%m%d')
    and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y%m%d')
        and service_obj_service_name in ('Link','Auto')
    and order_id is not null
    and service_obj_city_display_name is not null
),

order_coin as (
select
    week(date_parse(yyyymmdd,'%Y%m%d')) week_number,
    date_format(date_trunc('week',date_parse(yyyymmdd,'%Y%m%d')), '%Y-%m-%d') week_date,
    owner_id customerid,
    entity_id,
    cast(json_extract(coin_wallet_changes, '$[0].offerType') as varchar) offerType,
    coalesce(round(sum(case when transaction_type = 'credit' then cast(amount as double) end)),0) as coin_credited,
    coalesce(round(sum(case when transaction_type = 'debit' and subtype != 'coinExpired' then cast(amount as double) end)),0) as coin_utilized,
    coalesce(round(sum(case when transaction_type = 'debit' and subtype = 'coinExpired' then cast(amount as double) end)),0) as coin_expired

from payments.transactions_snapshot
where 
    yyyymmdd between  date_format(date_trunc('week', cast('{start_date}' as date)),'%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y%m%d')
    and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y%m%d')
    and owner_type ='customer'
    and transaction_status = 'done'
    and json_extract(coin_wallet_changes, '$[0].id') is not null 
group by 1,2,3,4,5
),

service_total_coins as (
select
    week_date,
    week_number,
    customer_id,
    entity_id,
    sum(case when offerType in ('locationOffer','rideOffer','scratchCardOffer') and service_obj_service_name in ('Auto','Link') then coin_credited end) total_coin_credited,
    sum(case when offerType in ('locationOffer','rideOffer','scratchCardOffer') and service_obj_service_name in ('Auto','Link') then coin_expired end) total_coin_expired,
    sum(case when offerType = 'giftOffer' and service_obj_service_name in ('Auto','Link') then coin_utilized end) giftOffer_utilized,
    sum(case when offerType = 'walletRechargeOffer' and service_obj_service_name in ('Auto','Link') then coin_utilized end) walletOffer_utilized,
    sum(case when offerType = 'dashboardCoinsCredit' and service_obj_service_name in ('Auto','Link') then coin_utilized end) dashboardCoinsCredit_utilized
from
    (
    select
        a.*,
        b.*
    from order_coin a 
    join orders b on a.customerid = b.customer_id and a.entity_id = b.order_id and a.week_date = b.start_date
    )
where week_date is not null
group by 1,2,3,4
),

coin_credit as
(
select
    start_date,
    week_number,
    customerid,
    coalesce(lifetime_stage, '0. NEW') lifetime_stage,
    sum(total_coin_credited) total_coin_credited,
    sum(walletOffer_utilized) walletOffer_utilized,
    sum(giftOffer_utilized) giftOffer_utilized,
    sum(dashboardCoinsCredit_utilized) dashboardCoinsCredit_utilized
from
    (
    select
        (case when a.week is null then b.week_number else a.week end) as week,
        (case when a.customer_id is null then b.customer_id else a.customer_id end) as customerid,
        (case when a.start_date is null then b.week_date else a.start_date end) as start_date,
        a.lifetime_stage,
        b.week_number,
        b.walletOffer_utilized,
        b.giftOffer_utilized,
        b.dashboardCoinsCredit_utilized,
        b.total_coin_credited
    from rf_daily_kpi a 
    right join service_total_coins b on a.week = b.week_number and a.customer_id = b.customer_id
    )
where start_date is not null
group by 1,2,3,4
),

coin_expired_customer_level as (
select
    week_date as Start2,
    customerid as cust2,
    sum(coin_expired) coin_expired
from order_coin a 
join orders b on a.entity_id = b.order_id
where coin_expired > 0
group by 1,2
),

coin_expired_final as (
select
    a.*,    
    coalesce(lifetime_stage, '0. NEW') lifetime_stage
from coin_expired_customer_level a 
left join rf_daily_kpi b on a.cust2 = b.customer_id and a.Start2 = b.start_date
),

agg_coin_expire as (
select
    Start2,
    cust2,
    lifetime_stage,
    sum(coin_expired) coin_expired
from coin_expired_final
group by 1,2,3
),

agg_coin as (
select
    a.*,
    b.coin_expired,
    (coalesce(a.total_coin_credited,0) + coalesce(a.walletOffer_utilized,0) + coalesce(giftOffer_utilized,0) + coalesce(a.dashboardCoinsCredit_utilized,0) - coalesce(b.coin_expired,0)) total_coin_burn
from coin_credit a 
left join agg_coin_expire b on a.start_date = b.Start2 and customerid = cust2  
),

ride_with_coin as (
select 
    a.*,
    b.total_coin_burn as total_coin_burn,
    (a.burn + b.total_coin_burn) as discount_with_coin
from rf_daily_kpi a 
join agg_coin b on a.week_number = b.week_number and a.customer_id = b.customerid --and a.lifetime_stage = b.lifetime_stage
)

select 
    week_number,
    lifetime_stage,
    count(distinct customer_id) as base,
    count(distinct case when net_rides > 0 then customerid end) as net_cust,
    count(distinct case when net_rides > 0 then customerid end)*100.00/count(distinct customer_id)  "net convr %",
    sum(net_rides) as net_rides,
    sum(net_rides)*100/sum(gross_rides) "G2N %",
    sum(net_rides)*1.00/count(distinct case when net_rides > 0 then customerid end) RPC,
    sum(sub_rides) as subs_rides,
    sum(subs_discount) as subs_discount,
    sum(burn) as discount,
    sum(total_coin_burn) total_coin_burn,
    sum(discount_with_coin) discount_with_coin,
    sum(burn)*100.0/sum(subtotal) as discount_perc_without_coins,
    sum(discount_with_coin)*100.0/sum(subtotal) as discount_perc_with_coins,
    sum(subtotal) as subtotal,
    sum(subtotal)*1.0/sum(net_rides) as ATV,
    sum(burn)*1.0/sum(net_rides) as DPR,
    100.0*count(distinct case when ao > 0 then customerid end)/count(distinct customer_id) as "ao_cust %",
    100.0*count(distinct case when fe > 0 then customerid end)/count(distinct customer_id) as "fe_cust %",
    100.0*count(distinct case when rr > 0 then customerid end)/count(distinct customer_id) as "rr_cust %",
    sum(ao) as ao_sessions,
    sum(fe) as fe_sessions,
    sum(rr) as rr_sessions,
    100*sum(rr)/cast(sum(fe) as real) as "fe-rr %"

from ride_with_coin
group by 1,2
order by 1 desc, 2
"""

In [14]:
df_pan_ls_taxi = pd.read_sql(pan_taxi, conn)
df_pan_ls_taxi


,week_number,lifetime_stage,base,net_cust,net convr %,net_rides,G2N %,RPC,subs_rides,subs_discount,...,subtotal,ATV,DPR,ao_cust %,fe_cust %,rr_cust %,ao_sessions,fe_sessions,rr_sessions,fe-rr %
0,45,1.HANDHOLDING,52686,52466,99.58,104780,59,2.00,531,7497.0,...,7159696.0,68.330750,10.112980,100.0,100.0,100.0,530418,411999,141529,34.351784
1,45,2.HOOK,32547,32330,99.33,75835,61,2.35,1364,17225.0,...,5034409.0,66.386352,7.571042,100.0,100.0,100.0,356057,281127,99978,35.563286
2,45,3.DETOX,17542,17477,99.63,50691,60,2.90,3868,53971.0,...,2755882.0,54.366298,7.279083,100.0,100.0,100.0,241805,195139,67520,34.600975
3,45,4.SUSTENANCE,8682,8609,99.16,20459,55,2.38,336,4119.0,...,1593353.0,77.880297,8.140134,100.0,100.0,100.0,99968,77939,29335,37.638410
4,45,5.COMMITTED,16751,16671,99.52,69754,55,4.18,3856,49798.0,...,4907485.0,70.354173,4.676821,100.0,100.0,100.0,300719,232338,97554,41.987965
5,45,6.CHURN_OTB,7751,7699,99.33,18458,60,2.40,413,5680.0,...,1241881.0,67.281450,8.090151,100.0,100.0,100.0,84077,66711,24755,37.107822
6,45,7.SOFT_CHURN,3538,3515,99.35,6900,65,1.96,104,1451.0,...,475561.0,68.921884,10.456232,100.0,100.0,100.0,31809,24973,8852,35.446280
7,45,8.DORMANT,65247,64959,99.56,114050,63,1.76,875,11675.0,...,8242890.0,72.274353,11.989391,100.0,100.0,100.0,542741,425597,148220,34.826374
8,45,9.INACTIVE,38402,38177,99.41,64124,65,1.68,79,936.0,...,4826745.0,75.272051,12.642911,100.0,100.0,100.0,303768,230009,81455,35.413830
9,44,1.HANDHOLDING,49266,49116,99.70,96476,56,1.96,585,8502.0,...,6661907.0,69.052479,10.191623,100.0,100.0,100.0,496105,387746,132191,34.092163


In [18]:
start_date = '2022-11-07'
end_date = '2022-11-13'
service = 'taxi'

In [21]:
## base
base = f"""
with base as (
select
    run_date,
    week,
    case 
    when lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
    when lifetime_stage = 'HOOK' then '2.HOOK'
    when lifetime_stage = 'DETOX' then '3.DETOX'
    when lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
    when lifetime_stage = 'COMMITTED' then '5.COMMITTED'
    when lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
    when lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
    when lifetime_stage = 'DORMANT' then '8.DORMANT'
    when lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
    end lifetime_stage,
    customer_id
from 
    (
    select
        run_date,
        (extract(week from cast(run_date as date)) + 1) as week,
        case
        when lower('{service}') = ('link') then link_lifetime_stage 
        when lower('{service}') = ('auto') then auto_lifetime_stage 
        else taxi_lifetime_stage end lifetime_stage,
        customer_id,
        count(*) records
    from datasets.iallocator_customer_segments 
    where 
        run_date between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '1' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '15' day), '%Y-%m-%d')
        and  date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and day_of_week(cast (run_date as date)) = 7
        and case
            when lower('{service}') = ('link') then link_lifetime_rides 
            when lower('{service}') = ('auto') then auto_lifetime_rides 
            else taxi_lifetime_rides end > 0 
    group by 1,2,3,4
    )
)

select week, lifetime_stage, count(distinct customer_id) base from base group by 1,2 order by 2
"""

df_base = pd.read_sql(base,conn)
df_base

,week,lifetime_stage,base
0,45,1.HANDHOLDING,1435293
1,45,2.HOOK,1396420
2,45,3.DETOX,280707
3,45,4.SUSTENANCE,546920
4,45,5.COMMITTED,731851
5,45,6.CHURN_OTB,366298
6,45,7.SOFT_CHURN,255471
7,45,8.DORMANT,6538406
8,45,9.INACTIVE,16943555


In [56]:
## rf_daily_kpi

rf_daily_kpi = f"""
with base as (
select
    run_date,
    week,
    case 
    when lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
    when lifetime_stage = 'HOOK' then '2.HOOK'
    when lifetime_stage = 'DETOX' then '3.DETOX'
    when lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
    when lifetime_stage = 'COMMITTED' then '5.COMMITTED'
    when lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
    when lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
    when lifetime_stage = 'DORMANT' then '8.DORMANT'
    when lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
    end lifetime_stage,
    customer_id
from 
    (
    select
        run_date,
        (extract(week from cast(run_date as date)) + 1) as week,
        case
        when lower('{service}') = ('link') then link_lifetime_stage 
        when lower('{service}') = ('auto') then auto_lifetime_stage 
        else taxi_lifetime_stage end lifetime_stage,
        customer_id,
        count(*) records
    from datasets.iallocator_customer_segments 
    where 
        run_date between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '1' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '15' day), '%Y-%m-%d')
        and  date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and day_of_week(cast (run_date as date)) = 7
        and case
            when lower('{service}') = ('link') then link_lifetime_rides 
            when lower('{service}') = ('auto') then auto_lifetime_rides 
            else taxi_lifetime_rides end > 0 
    group by 1,2,3,4
    )
),

rf_daily_kpi as (
select
    base.lifetime_stage,
    base.week week_number,
    base.customer_id,
    a.*
from     
    (
        select
            customerid,
            extract(week from cast(day as date)) as week,
            date_format(date_trunc('week', cast(day as date)), '%Y-%m-%d') start_date,
            sum(gross_rides_daily) as gross_rides,
            sum(case when net_rides_daily > 0 then net_rides_daily end) as net_rides,
            sum(case when net_rides_daily > 0 then subscription_rides_daily end) as sub_rides,
            sum(case when net_rides_daily > 0 then burn_daily end) as burn,
            sum(case when subscription_rides_daily > 0 then burn_daily end) as subs_discount,
            sum(case when net_rides_daily > 0 then subtotal_daily end) as subtotal,
            sum(case
                when ao_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when ao_sessions_unique_daily = 0 and fe_sessions_unique_daily > 0 then fe_sessions_unique_daily
                when ao_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else ao_sessions_unique_daily end 
                ) as ao,
            sum(case
                when fe_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when fe_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else fe_sessions_unique_daily end 
                ) as fe,
            sum(case 
                when rr_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                else rr_sessions_unique_daily end ) as rr
        from 
        datasets.customer_rf_daily_kpi
        where
        day between  date_format(date_trunc('week', cast('{start_date}' as date)), '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y-%m-%d')
        and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and service_name in ('Auto', 'Link')
        group by 1,2,3
    ) as a
right join base on base.customer_id = a.customerid and base.week = a.week
)

select week_number, lifetime_stage, count(distinct customer_id) base, count(distinct customerid) from rf_daily_kpi group by 1,2 order by 2
"""

df_rf_daily_kpi = pd.read_sql(rf_daily_kpi, conn)
df_rf_daily_kpi

,week_number,lifetime_stage,base,_col3
0,45,1.HANDHOLDING,1435293,429212
1,45,2.HOOK,1396420,613185
2,45,3.DETOX,280707,189210
3,45,4.SUSTENANCE,546920,309812
4,45,5.COMMITTED,731851,597207
5,45,6.CHURN_OTB,366298,185254
6,45,7.SOFT_CHURN,255471,88039
7,45,8.DORMANT,6538406,1168975
8,45,9.INACTIVE,16943555,578421


In [41]:
## without coin

without_coin = f"""
with base as (
select
    run_date,
    week,
    case 
    when lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
    when lifetime_stage = 'HOOK' then '2.HOOK'
    when lifetime_stage = 'DETOX' then '3.DETOX'
    when lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
    when lifetime_stage = 'COMMITTED' then '5.COMMITTED'
    when lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
    when lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
    when lifetime_stage = 'DORMANT' then '8.DORMANT'
    when lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
    end lifetime_stage,
    customer_id
from 
    (
    select
        run_date,
        (extract(week from cast(run_date as date)) + 1) as week,
        case
        when lower('{service}') = ('link') then link_lifetime_stage 
        when lower('{service}') = ('auto') then auto_lifetime_stage 
        else taxi_lifetime_stage end lifetime_stage,
        customer_id,
        count(*) records
    from datasets.iallocator_customer_segments 
    where 
        run_date between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '1' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '15' day), '%Y-%m-%d')
        and  date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and day_of_week(cast (run_date as date)) = 7
        and case
            when lower('{service}') = ('link') then link_lifetime_rides 
            when lower('{service}') = ('auto') then auto_lifetime_rides 
            else taxi_lifetime_rides end > 0 
    group by 1,2,3,4
    )
),

rf_daily_kpi as (
select
    base.lifetime_stage,
    base.week week_number,
    base.customer_id,
    a.*
from     
    (
        select
            customerid,
            extract(week from cast(day as date)) as week,
            date_format(date_trunc('week', cast(day as date)), '%Y-%m-%d') start_date,
            sum(gross_rides_daily) as gross_rides,
            sum(case when net_rides_daily > 0 then net_rides_daily end) as net_rides,
            sum(case when net_rides_daily > 0 then subscription_rides_daily end) as sub_rides,
            sum(case when net_rides_daily > 0 then burn_daily end) as burn,
            sum(case when subscription_rides_daily > 0 then burn_daily end) as subs_discount,
            sum(case when net_rides_daily > 0 then subtotal_daily end) as subtotal,
            sum(case
                when ao_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when ao_sessions_unique_daily = 0 and fe_sessions_unique_daily > 0 then fe_sessions_unique_daily
                when ao_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else ao_sessions_unique_daily end 
                ) as ao,
            sum(case
                when fe_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when fe_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else fe_sessions_unique_daily end 
                ) as fe,
            sum(case 
                when rr_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                else rr_sessions_unique_daily end ) as rr
        from 
        datasets.customer_rf_daily_kpi
        where
        day between  date_format(date_trunc('week', cast('{start_date}' as date)), '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y-%m-%d')
        and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and service_name in ('Auto', 'Link')
        group by 1,2,3
    ) as a
right join base on base.customer_id = a.customerid and base.week = a.week
)

select 
    week_number,
    lifetime_stage,
    count(distinct customer_id) as base,
    count(distinct case when net_rides > 0 then customerid end) as net_cust,
    count(distinct case when net_rides > 0 then customerid end)*100.00/count(distinct customer_id)  "net convr %",
    sum(net_rides) as net_rides,
    sum(net_rides)*100/sum(gross_rides) "G2N %",
    sum(net_rides)*1.00/count(distinct case when net_rides > 0 then customerid end) RPC,
    sum(sub_rides) as subs_rides,
    sum(subs_discount) as subs_discount,
    sum(burn) as discount,
    sum(burn)*100.0/sum(subtotal) as discount_perc_without_coins,
    sum(subtotal) as subtotal,
    sum(subtotal)*1.0/sum(net_rides) as ATV,
    sum(burn)*1.0/sum(net_rides) as DPR,
    100.0*count(distinct case when ao > 0 then customerid end)/count(distinct customer_id) as "ao_cust %",
    100.0*count(distinct case when fe > 0 then customerid end)/count(distinct customer_id) as "fe_cust %",
    100.0*count(distinct case when rr > 0 then customerid end)/count(distinct customer_id) as "rr_cust %",
    sum(ao) as ao_sessions,
    sum(fe) as fe_sessions,
    sum(rr) as rr_sessions,
    100*sum(rr)/cast(sum(fe) as real) as "fe-rr %"

from rf_daily_kpi
group by 1,2
order by 1 desc, 2
"""

df_without_coin = pd.read_sql(without_coin, conn)
df_without_coin

,week_number,lifetime_stage,base,net_cust,net convr %,net_rides,G2N %,RPC,subs_rides,subs_discount,...,subtotal,ATV,DPR,ao_cust %,fe_cust %,rr_cust %,ao_sessions,fe_sessions,rr_sessions,fe-rr %
0,45,1.HANDHOLDING,1435293,208408,14.52,351907,41,1.69,2532,34000.0,...,30166750.0,85.723643,5.260967,29.8,29.8,20.7,2702649,2173230,642905,29.582926
1,45,2.HOOK,1396420,369557,26.46,718554,46,1.94,22662,282383.0,...,57093252.0,79.455757,3.061912,43.8,43.8,34.4,4169693,3378697,1182083,34.986360
2,45,3.DETOX,280707,140400,50.02,418581,58,2.98,84046,1187921.0,...,21862310.0,52.229580,7.152632,67.4,67.4,57.4,1962293,1600913,573113,35.799133
3,45,4.SUSTENANCE,546920,187179,34.22,340435,37,1.82,5431,67819.0,...,31889758.0,93.673559,1.851631,56.6,56.6,47.3,2230559,1789196,668511,37.363766
4,45,5.COMMITTED,731851,494389,67.55,1715892,44,3.47,125689,1623736.0,...,132576537.0,77.263917,1.939641,81.6,81.6,76.9,7918543,6251911,2800092,44.787780
5,45,6.CHURN_OTB,366298,117598,32.10,238262,45,2.03,9843,129895.0,...,18844945.0,79.093372,3.366277,50.5,50.5,41.3,1326494,1073378,396829,36.970108
6,45,7.SOFT_CHURN,255471,48349,18.93,78563,47,1.62,1201,14835.0,...,6413492.0,81.635019,4.262286,34.3,34.3,25.3,484109,395073,127574,32.291250
7,45,8.DORMANT,6538406,513831,7.86,765644,40,1.49,7327,87393.0,...,66921754.0,87.405836,4.042878,17.8,17.8,11.9,5946792,4853966,1457009,30.016876
8,45,9.INACTIVE,16943555,216537,1.28,312834,42,1.44,1932,25389.0,...,26513724.0,84.753332,5.416339,3.4,3.4,2.0,2644463,2120636,574769,27.103615


In [109]:
## coin logic 

coin = f"""
with base as (
select
    run_date,
    week,
    case 
    when lifetime_stage = 'HANDHOLDING' then '1.HANDHOLDING'
    when lifetime_stage = 'HOOK' then '2.HOOK'
    when lifetime_stage = 'DETOX' then '3.DETOX'
    when lifetime_stage = 'SUSTENANCE' then '4.SUSTENANCE'
    when lifetime_stage = 'COMMITTED' then '5.COMMITTED'
    when lifetime_stage = 'CHURN_OTB' then '6.CHURN_OTB'
    when lifetime_stage = 'SOFT_CHURN' then '7.SOFT_CHURN'
    when lifetime_stage = 'DORMANT' then '8.DORMANT'
    when lifetime_stage in ('INACTIVE','UNKNOWN') then '9.INACTIVE' 
    end lifetime_stage,
    customer_id
from 
    (
    select
        run_date,
        (extract(week from cast(run_date as date)) + 1) as week,
        case
        when lower('{service}') = ('link') then link_lifetime_stage 
        when lower('{service}') = ('auto') then auto_lifetime_stage 
        else taxi_lifetime_stage end lifetime_stage,
        customer_id,
        count(*) records
    from datasets.iallocator_customer_segments 
    where 
        run_date between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '1' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '15' day), '%Y-%m-%d')
        and  date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and day_of_week(cast (run_date as date)) = 7
        and case
            when lower('{service}') = ('link') then link_lifetime_rides 
            when lower('{service}') = ('auto') then auto_lifetime_rides 
            else taxi_lifetime_rides end > 0 
    group by 1,2,3,4
    )
),

rf_daily_kpi as (
select
    base.lifetime_stage,
    base.week week_number,
    base.customer_id,
    a.*
from     
    (
        select
            customerid,
            extract(week from cast(day as date)) as week,
            date_format(date_trunc('week', cast(day as date)), '%Y-%m-%d') start_date,
            sum(gross_rides_daily) as gross_rides,
            sum(case when net_rides_daily > 0 then net_rides_daily end) as net_rides,
            sum(case when net_rides_daily > 0 then subscription_rides_daily end) as sub_rides,
            sum(case when net_rides_daily > 0 then burn_daily end) as burn,
            sum(case when subscription_rides_daily > 0 then burn_daily end) as subs_discount,
            sum(case when net_rides_daily > 0 then subtotal_daily end) as subtotal,
            sum(case
                when ao_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when ao_sessions_unique_daily = 0 and fe_sessions_unique_daily > 0 then fe_sessions_unique_daily
                when ao_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else ao_sessions_unique_daily end 
                ) as ao,
            sum(case
                when fe_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                when fe_sessions_unique_daily = 0 and rr_sessions_unique_daily > 0 then rr_sessions_unique_daily
                else fe_sessions_unique_daily end 
                ) as fe,
            sum(case 
                when rr_sessions_unique_daily = 0 and net_rides_daily > 0 then net_rides_daily
                else rr_sessions_unique_daily end ) as rr
        from 
        datasets.customer_rf_daily_kpi
        where
        day between  date_format(date_trunc('week', cast('{start_date}' as date)), '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y-%m-%d')
        and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y-%m-%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y-%m-%d')
        and service_name in ('Auto', 'Link')
        group by 1,2,3
    ) as a
right join base on base.customer_id = a.customerid and base.week = a.week
),

orders as (
select
    week(date_parse(yyyymmdd,'%Y%m%d')) week,
    date_format(date_trunc('week',date_parse(yyyymmdd,'%Y%m%d')), '%Y-%m-%d') start_date,
    customer_id,
    order_id,
    city_name,
    discount,
    amount,
    sub_total,
    service_obj_service_name,
    order_status
from orders.order_logs_snapshot
where 
    yyyymmdd between  date_format(date_trunc('week', cast('{start_date}' as date)) - interval '14' day,'%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '28' day), '%Y%m%d')
    and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y%m%d')
        and service_obj_service_name in ('Link','Auto')
    and order_id is not null
    and service_obj_city_display_name is not null
),

order_coin as (
select
    week(date_parse(yyyymmdd,'%Y%m%d')) week_number,
    date_format(date_trunc('week',date_parse(yyyymmdd,'%Y%m%d')), '%Y-%m-%d') week_date,
    owner_id customerid,
    entity_id,
    cast(json_extract(coin_wallet_changes, '$[0].offerType') as varchar) offerType,
    coalesce(round(sum(case when transaction_type = 'credit' then cast(amount as double) end)),0) as coin_credited,
    coalesce(round(sum(case when transaction_type = 'debit' and subtype != 'coinExpired' then cast(amount as double) end)),0) as coin_utilized,
    coalesce(round(sum(case when transaction_type = 'debit' and subtype = 'coinExpired' then cast(amount as double) end)),0) as coin_expired

from payments.transactions_snapshot
where 
    yyyymmdd between  date_format(date_trunc('week', cast('{start_date}' as date)),'%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE) - INTERVAL '14' day), '%Y%m%d')
    and  date_format(date_trunc('week', cast('{end_date}' as date) + INTERVAL '7' day) - INTERVAL '1' day, '%Y%m%d') --]] date_format((date_trunc('week', CURRENT_DATE)), '%Y%m%d')
    and owner_type ='customer'
    and transaction_status = 'done'
    and json_extract(coin_wallet_changes, '$[0].id') is not null 
group by 1,2,3,4,5
),

service_total_coins as (
select
    week_date,
    week_number,
    customer_id,
    entity_id,
    sum(case when offerType in ('locationOffer','rideOffer','scratchCardOffer') and service_obj_service_name in ('Auto','Link') then coin_credited end) total_coin_credited,
    sum(case when offerType in ('locationOffer','rideOffer','scratchCardOffer') and service_obj_service_name in ('Auto','Link') then coin_expired end) total_coin_expired,
    sum(case when offerType = 'giftOffer' and service_obj_service_name in ('Auto','Link') then coin_utilized end) giftOffer_utilized,
    sum(case when offerType = 'walletRechargeOffer' and service_obj_service_name in ('Auto','Link') then coin_utilized end) walletOffer_utilized,
    sum(case when offerType = 'dashboardCoinsCredit' and service_obj_service_name in ('Auto','Link') then coin_utilized end) dashboardCoinsCredit_utilized
from
    (
    select
        a.*,
        b.*
    from order_coin a 
    join orders b on a.customerid = b.customer_id and a.entity_id = b.order_id --and a.week_date = b.start_date
    )
where week_date is not null
group by 1,2,3,4
),

coin_credit as
(
select
    start_date,
    week_number,
    coalesce(lifetime_stage, '0. NEW') lifetime_stage,
    sum(total_coin_credited) total_coin_credited,
    sum(walletOffer_utilized) walletOffer_utilized,
    sum(giftOffer_utilized) giftOffer_utilized,
    sum(dashboardCoinsCredit_utilized) dashboardCoinsCredit_utilized
from
    (
    select
        (case when a.week_number is null then b.week_number else a.week_number end) as week,
        (case when a.customer_id is null then b.customer_id else a.customer_id end) as customerid,
        (case when a.start_date is null then b.week_date else a.start_date end) as start_date,
        a.lifetime_stage,
        b.week_number,
        b.walletOffer_utilized,
        b.giftOffer_utilized,
        b.dashboardCoinsCredit_utilized,
        b.total_coin_credited
    from rf_daily_kpi a 
    left join service_total_coins b on a.week_number = b.week_number and a.customer_id = b.customer_id
    )
where start_date is not null
group by 1,2,3
),

coin_expired_customer_level as (
select
    start_date as Start2,
    customerid as cust2,
    sum(coin_expired) coin_expired
from order_coin a 
join orders b on a.entity_id = b.order_id
where coin_expired > 0
group by 1,2
),

coin_expired_final as (
select
    a.Start2,    
    coalesce(lifetime_stage, '0. NEW') lifetime_stage,
    sum(coin_expired) coin_expired
from coin_expired_customer_level a 
left join rf_daily_kpi b on a.cust2 = b.customer_id and a.Start2 = b.start_date
group by 1,2
),

agg_coin as (
select
    a.*,
    b.coin_expired,
    (coalesce(a.total_coin_credited,0) + coalesce(a.walletOffer_utilized,0) + coalesce(giftOffer_utilized,0) + coalesce(a.dashboardCoinsCredit_utilized,0) - coalesce(b.coin_expired,0)) total_coin_burn
from coin_credit a 
join coin_expired_final b on a.start_date = b.Start2 and a.lifetime_stage = b.lifetime_stage
)

select
    b.week_number,
    b.lifetime_stage,
    count(distinct customer_id) as base,
    count(distinct case when net_rides > 0 then customerid end) as net_cust,
    count(distinct case when net_rides > 0 then customerid end)*100.00/count(distinct customer_id)  "net convr %",
    sum(net_rides) as net_rides,
    sum(net_rides)*100/sum(gross_rides) "G2N %",
    sum(net_rides)*1.00/count(distinct case when net_rides > 0 then customerid end) RPC,
    sum(sub_rides) as subs_rides,
    sum(subs_discount) as subs_discount,
    sum(burn) as discount,
    --sum(total_coin_burn) total_coin_burn,
    --sum(a.burn)+sum(b.total_coin_burn) discount_with_coin,
    sum(burn)*100.0/sum(subtotal) as discount_perc_without_coins,
    --sum(discount_with_coin)*100.0/sum(subtotal) as discount_perc_with_coins,
    sum(subtotal) as subtotal,
    sum(subtotal)*1.0/sum(net_rides) as ATV,
    sum(burn)*1.0/sum(net_rides) as DPR,
    100.0*count(distinct case when ao > 0 then customerid end)/count(distinct customer_id) as "ao_cust %",
    100.0*count(distinct case when fe > 0 then customerid end)/count(distinct customer_id) as "fe_cust %",
    100.0*count(distinct case when rr > 0 then customerid end)/count(distinct customer_id) as "rr_cust %",
    sum(ao) as ao_sessions,
    sum(fe) as fe_sessions,
    sum(rr) as rr_sessions,
    100*sum(rr)/cast(sum(fe) as real) as "fe-rr %"

from rf_daily_kpi a
left join agg_coin b on a.week_number = b.week_number and a.lifetime_stage = b.lifetime_stage
group by 1,2
order by 1 desc, 2

"""

df_coin = pd.read_sql(coin, conn)
df_coin

,week_number,lifetime_stage,base,net_cust,net convr %,net_rides,G2N %,RPC,subs_rides,subs_discount,...,subtotal,ATV,DPR,ao_cust %,fe_cust %,rr_cust %,ao_sessions,fe_sessions,rr_sessions,fe-rr %
0,45.0,1.HANDHOLDING,1435293,208408,14.52,351907,41,1.69,2532,34000.0,...,30166750.0,85.723643,5.260967,29.8,29.8,20.7,2702649,2173230,642905,29.582926
1,NaN,None,27059628,2087840,7.72,4588765,44,2.20,258131,3419371.0,...,362115772.0,78.913558,3.286159,13.7,13.7,10.3,26682946,21463770,7779980,36.247036


In [106]:
df_coin['lifetime_stage'].unique()

array(['1.HANDHOLDING'], dtype=object)